In [ ]:
# inspired by
# https://github.com/fjg00/Facebook-Group-Post-Scraper/blob/main/Facebook%20Group%20Parser.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import argparse
from datetime import datetime
import time
import json

import os
import re
import pandas as pd

In [ ]:
# USER DEFINED INPUTS
import Login
CHROME_DRIVER_PATH = './chromedriver' # must download this yourself

# EMAIL, PASSWORD from Login file (explained in README.md)
EMAIL = Login.EMAIL
PASSWORD = Login.PASSWORD

# Group number
GROUP_IDS = [1379345962387168, "kamer.in.eindhoven"] #USER INPUT
GROUP_URLS = []
for id in GROUP_IDS:
    item = 'https://www.facebook.com/groups/'+str(id)
    GROUP_URLS.append(item)

SEARCH_PROMPT = "" # something to be searched for in the group
SEARCH_PROMPT = SEARCH_PROMPT.replace(" ","%20")

# TODO populate this
# Copy the group description and put it here to be able to filter it out
GROUP_DESCRIPTIONS = ["""-Kamer in Eindhoven-
Kamer in Eindhoven is een platform waar vraag escraper.posts[0]n aanbod (kosteloos) worden samengebracht. Ben je opzoek naar een kamer, huisgenoot of bied je een kamer aan, word dan nu lid van deze facebookgroep.
Je scrollt door de nieuwste kamers en huisgenoten om vervolgens direct te reageren naar de aanbieder. Bij ons geen inschrijfkosten, gewoon studenten onder elkaar.
Kamer in Eindhoven is onderdeel van de Facebook community Zoekt Kamer in… Wij zijn ook actief in Nijmegen, Amsterdam, Delft, Rotterdam, Groningen, Maastricht, Breda, Utrecht, Leiden, Den Haag, Haarlem en Amersfoort.
Sinds 2020 hebben wij een samenwerking met de gratis app MyHospi. Hiermee willen wij het proces voor jullie nog makkelijker maken.
HOE WERKT MYHOSPI VOOR HUIZEN?
1. Plaats de kamer op myHospi via de app.
2. Deel de link naar het huis in je FB bericht en geef aan dat mensen via myHospi (de link) moeten reageren. Met myHospi krijg je direct een compleet beeld van de mensen die reageren op jouw kamer. Geen onpersoonlijke mails maar complete profielen.
3. Swipe en Like!
4. Nodig kandidaten uit en deel de link naar je videocall."""] # User input

In [ ]:
from FBScraper import FBScraper
scraper = FBScraper(CHROME_DRIVER_PATH, Login.EMAIL, Login.PASSWORD)
scraper.openAndLogin(hide=True)
scraper.setGroupInfo(GROUP_IDS)
scraper.getPostURLs(limit=5, group_index=0)
print(scraper.posts[0])
test_profile = scraper.getPosterURL(scraper.posts[0])
test_profile

In [ ]:
scraper.closeSession()

In [ ]:
def addvalues(dictionary: dict, key, L: list):
    """Append multiple values to a key in the given dictionary"""
    if key not in dictionary:
        dictionary[key] = list()
    dictionary[key].extend(L)
    return dictionary


In [ ]:
# Opens FB group with search parameters
# driver.get(GROUP_URL+SEARCH_PROMPT)

In [ ]:
# # Selection criteria
# text = ""
# comments = ""
# z = 1

In [ ]:
def getPosterURL(post: str, group_id: int, page_source=None, soup=None):
    driver.get(post)
    time.sleep(1)
    # TODO something here is broken. Fix this.
    poster_class = "x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz xt0b8zv xzsf02u x1s688f"

    link = driver.page_source if page_source is None else page_source
    page_source = 'https://www.facebook.com' + page_source if link is None else link


    soup = BeautifulSoup(page_source, 'html.parser') if soup is None else soup

    urls = soup.find_all('a', class_=poster_class)
    urls = [url['href'] for url in urls]

    identifier = "/groups/" + str(group_id)
    try:
        url = [url for url in urls if identifier in url][0]
    except:
        url = [url for url in urls if identifier in url]    
    url = "https://www.facebook.com" + str(url)

    return url

In [ ]:
def getPosterList(post_list: list, group_url: str):
    urls = []
    for count, post in enumerate(post_list):
        urls.append(getPosterURL(post, group_url))
        if count % 10:
            print (count, ' posters gotten')
    return urls

In [ ]:
def getLastXPosters(X: int, group_url):
    tolerance_diff = 5
    posts = getPostURLs(X+tolerance_diff, group_url)
    
    now = datetime.now()
    now = now.strftime("%Y-%m-%d_%H-%M")
    
    # Save before continuing
    pdf = pd.DataFrame({'post': posts})
    pdf.to_csv(f'{now}_{X}_posts.csv', index=False)

    posters = getPosterList(posts, group_url)
    
    pdf = pd.DataFrame({'profiles': posters})
    pdf.to_csv(f'{now}_{X}_profiles.csv', index=False)

    return posters, posts

In [ ]:
posters, posts = getLastXPosters(60)

In [ ]:

# driver.quit()

In [ ]:
def message(profile: str, msg: str):
    # Add enter to message so that it gets sent
    msg_end = msg[len(msg)-2] + msg[len(msg)-1]
    if msg_end != "\n":
        msg = msg + "\n"

    # Open the profile
    driver.get(profile)
    
    # Click the "Message" button
    xpath_msg_btn = "//span[contains(text(),'Message')]"
    page_source = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_msg_btn))).click()

    # Write something in the input
    xpath_input = "//p[@class='xdj266r xat24cr']"
    input_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_input)))
    input_field.send_keys(msg)


In [ ]:
def getName(profile: str):
    # Open the profile link
    driver.get(profile)
    name_selector = ".x14z4hjw > div:nth-child(1) > h1"
    name = driver.find_element(By.CSS_SELECTOR, name_selector)
    return name.text

In [ ]:
# Run the scraper
posters = []
posts = []
for group in GROUP_URLS:
    driver.get(group)
    posters_, posts_ = getLastXPosters(10, group)
    posters.extend(posters_)
    posts.extend(posts_)

In [ ]:
# # Get names
# names = []
# for profile in posters:
#     print(profile)
#     # names.extend(getName(profile))


In [ ]:
name_df = pd.DataFrame({'names': names, 'profiles': posters, 'posts': posts})

In [ ]:
# # Grab post and comment data
# # TODO make nice df out of it
# # TODO grab info about posters and commenters
# # TODO grab time data (and time difference between post and comment)


# WrapperDict = list() 
# texts = []
# translatedBools = []
# commentCounts = []

# for postCount, a in enumerate(posts):
#         driver.get(a)
#         time.sleep(1)
        
#         page_source = driver.page_source
#         soup = BeautifulSoup(page_source, 'html.parser')
        
#         # Relevant classes for text (or translate button)
#         translateBtnText = "x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x1n2onr6 x87ps6o x1a2a7pz xt0b8zv"
#         translateBtn = soup.find('div', class_ = translateBtnText)
#         translatedTextClass = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x6prxxf xvq8zen xo1l8bm xzsf02u x1yc453h"
#         originalTextClass = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x41vudc x6prxxf xvq8zen xo1l8bm xzsf02u x1yc453h"

#         if (translateBtn is not None) and (soup.find('div', class_ = translateBtnText).text == "See Translation"):
#                 isTranslated = False
#                 # Original
#                 # text = soup.find('span' , class_ = originalTextClass).text #post
#                 t = soup.find('div', class_ = "x1swvt13 x1l90r2v x1pi30zi x1iorvi4")
#                 if t is not None:
#                         text = t.text #post
#                 else:
#                         # This may result in text="Facebook", could find all of this class and choose one where it doesn't equal Facebook
#                         text = soup.find('span' , class_ = originalTextClass).text #post
#         elif soup.find('span', class_ = translatedTextClass) is not None:
#                 # Translated
#                 text = soup.find('span' , class_ = translatedTextClass).text #post
#                 isTranslated = True
#         elif soup.find('span', class_ = originalTextClass) is not None:
#                 # Original English
#                 text = soup.find('span', class_ = originalTextClass).text #post
#                 isTranslated = False

#         # isTranslated = len(soup.find_all('span', class_ = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1pg5gke xvq8zen xo1l8bm x1qq9wsj x1yc453h")) > 0
#         # TODO instead of length check that it equals expected text ('See original' or 'Rate this translation')

#         commentCount = soup.find('span', class_ = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x41vudc x6prxxf xvq8zen xo1l8bm xi81zsa").text
#         commentCount = int(re.search(r'\d+', commentCount).group())
        
#         # TODO create DF/dictionary/smth with all this info
#         texts.append(text)
#         translatedBools.append(isTranslated)
#         commentCounts.append(commentCount)
#         if postCount % 5 == 0:
#                 print (postCount)

#         # w = 1
        
#         # BranchDict = dict()
#         # BranchDict["tag"] = 1
#         # BranchDict["patterns"] = 1
#         # BranchDict["responses"] = list()
#         # L = list()
        
#         # for post in actualPosts:
#         #     s = post.get_text()
#         #     # if (s == "INSERT GROUP DESCRIPTION HERE"): #USER INPUT
#         #     if (s == GROUP_DESCRIPTION): #USER INPUT
#         #         time.sleep(1)
#         #     elif len(s.split()) < 4:
#         #         time.sleep(1)
#         #     else:
#         #         if w == 1: #post
#         #             text= text + post.get_text() 
#         #             BranchDict["tag"] = SEARCH_PROMPT + str(z) 
#         #             BranchDict["patterns"] = text
#         #             w = w + text1
#         #         elif w > 1 : #comments
#         #             comments = post.get_text()
#         #             L.append(comments)
#         #             comments = ""
#         # z = z + 1
#         # addvalues(BranchDict, "responses", L)
#         # WrapperDict.append(BranchDict)
#         # text = ""

In [ ]:
# Close the headless session (to avoid it staying alive in memory)
# driver.quit()

In [ ]:
# # df = pd.DataFrame([texts, translatedBools, commentCounts], columns=['Text', 'Is_Translated', 'Comment_Count'])
# df = pd.DataFrame.from_dict({"Text": texts, "Is_Translated": translatedBools, "Comment_Count": commentCounts, 'URL': list(posts)})

# # # Flatten text if necessary
# # def flatten(text):
# #     if text is list:
# #         text = ''.join(text)
# #         print (text)
# #     return text
# # df['Text'] = df['Text'].apply(flatten)
# # df.iloc[0]['URL']
# df

In [ ]:
# df.describe()

In [ ]:
# Save it, brother
# df.to_csv(path_or_buf='scraped_data.csv', index=False)